In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [ ]:
dataset = '/content/drive/MyDrive/hand data/keypoint.csv'
model_save_path = '/content/drive/MyDrive/hand data/keypoint_classifier.hdf5'
tflite_save_path = '/content/drive/MyDrive/hand data/keypoint_classifier.tflite'

In [ ]:
NUM_CLASSES = 7

In [ ]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
20/41 [=============>................] - ETA: 0s - loss: 1.9311 - accuracy: 0.1836 
Epoch 1: saving model to /content/drive/MyDrive/hand data/keypoint_classifier.hdf5
41/41 [==============================] - 1s 10ms/step - loss: 1.9018 - accuracy: 0.2054 - val_loss: 1.7976 - val_accuracy: 0.3360
Epoch 2/1000
22/41 [===============>..............] - ETA: 0s - loss: 1.8102 - accuracy: 0.2695

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: saving model to /content/drive/MyDrive/hand data/keypoint_classifier.hdf5
41/41 [==============================] - 0s 6ms/step - loss: 1.7942 - accuracy: 0.2801 - val_loss: 1.6885 - val_accuracy: 0.4212
Epoch 3/1000
26/41 [==================>...........] - ETA: 0s - loss: 1.7243 - accuracy: 0.3059
Epoch 3: saving model to /content/drive/MyDrive/hand data/keypoint_classifier.hdf5
41/41 [==============================] - 0s 4ms/step - loss: 1.7048 - accuracy: 0.3205 - val_loss: 1.5768 - val_accuracy: 0.4594
Epoch 4/1000
27/41 [==================>...........] - ETA: 0s - loss: 1.6448 - accuracy: 0.3414
Epoch 4: saving model to /content/drive/MyDrive/hand data/keypoint_classifier.hdf5
41/41 [==============================] - 0s 4ms/step - loss: 1.6271 - accuracy: 0.3447 - val_loss: 1.4695 - val_accuracy: 0.4774
Epoch 5/1000
26/41 [==================>...........] - ETA: 0s - loss: 1.5447 - accuracy: 0.3651
Epoch 5: saving model to /content/drive/MyDrive/hand data/keypoint_classifi

In [ ]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

14/14 [==============================] - 0s 2ms/step - loss: 0.2408 - accuracy: 0.9560


In [ ]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [ ]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 93ms/step
[8.9705443e-01 7.1029402e-02 3.1914573e-02 8.1682549e-07 7.5852483e-07
 4.1160142e-10 8.0879741e-14]
0


In [ ]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [ ]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

6676